In [32]:
%run '01-06_Homogeneous_Variety.ipynb'

class Irreducible_Equivariant_Vector_Bundle_Over_Irreducible_Homogeneous_Variety (object) :

    def __eq__ ( self , other ) :
        """Tests if two objects of the class ``Irreducible_Equivariant_Vector_Bundle_Over_Irreducible_Homogeneous_Variety`` coincide."""
        return type( other ) == self.__class__ and other.__repr__() == self.__repr__()


    def __init__( self , Base_Space , Highest_Weight ) :
        """
        Initialize a irreducible equivariant vector bundle over an irreducible homogeneous variety.

        INPUT:
        - ``BaseSpace`` -- Irreducible_Homogeneous_Variety;
        - ``Weights`` -- Highest weight space;

        OUTPUT: None.
        """
        assert type( Base_Space ) == Irreducible_Homogeneous_Variety , 'The input for ``Base_Space`` must be an irreducible homogeneous variety.'
        self._Base_Space = Base_Space
        assert Highest_Weight in self.Base_Space().Parent_Group().Cartan_Type().root_system().weight_space() , 'The input for ``Highest_Weight`` must be an element of the weight space.'
        self._Highest_Weight = Highest_Weight


    def __len__ ( self ) :
        """Returns the number of components of ``self``, namely 1, since it is irreducible by construction."""
        return 1


    def __mul__ ( self , other ) :
        """Tensor multiply to irred. equiv. vec. bdl.`s"""
        assert type(other) == self.__class__ , '``self`` and ``other`` need to be objects of the same class.'
        assert self.Base_Space() == other.Base_Space() , 'The base spaces need to coincide.'

        # Product of Twists, i.e. sum of the highest weights supported over the total twisting part
        Prd_Twist_HW =    self.Highest_Weight_Supported_Over( Nodes=self.Base_Space().Parabolic_Subgroup().Excluded_Nodes() ) \
                       + other.Highest_Weight_Supported_Over( Nodes=other.Base_Space().Parabolic_Subgroup().Excluded_Nodes() )

        # Product of Levi parts, i.e. product of the representations of highest weights supported over the total Levi part
        Prd_Levi_Rep =    self.Representation_Supported_Over_Levi_Part(Decomposition_Style='Coarse') \
                       * other.Representation_Supported_Over_Levi_Part(Decomposition_Style='Coarse')

        # In the remaining part, we merge the separate products over both the twisting part (excluded nodes) and the Levi part (included nodes)
        # 1. Define subroutine to get coefficients and multiplicities of a given weyl character
        def Get_Coefficients_And_Multiplicity_Of_Weyl_Character ( Weyl_Character ) :
            Simple_Coroots = WeylCharacterRing( Weyl_Character.cartan_type() ).simple_coroots()
            List = []
            for HighestWeight , Multiplicity in Weyl_Character :
                Coroots = [ HighestWeight.inner_product( Element ) for Element in list(Simple_Coroots) ]
                List += [ ( Coroots , Multiplicity ) ]
            return List
        # A product of two highest weights (= irreducible presentations) gives
        # in general a list of highest weights (= reducible presentations)
        n = self.Base_Space().Parent_Group().Cartan_Degree()
        e = { i : vector( QQ , (i-1)*[ 0 ] + [ 1 ] + (n-i)*[ 0 ] ) for i in [ 1 .. n ] }
            # ``e`` is Standard basis to write coefficients of weights in handsome vector-notation and
            # to apply base changes use by multiplication with appropriate Cartan matrix.
        Cartan_Matrix_From_C_To_B = self.Base_Space().Cartan_Matrix( From='C' , To='B' )
        Relabelling = self.Base_Space().Parabolic_Subgroup().Relabelling(Decomposition_Style='Coarse')
        mixed = self.Base_Space().Basis('mixed')
        for Prd_HW , Prd_Mult in Get_Coefficients_And_Multiplicity_Of_Weyl_Character( Prd_Levi_Rep ) :
            # 2. Write the coefficients of the current considered weight in terms of the basis C (fw's)
            C_Coefficients = vector( QQ , n*[ 0 ] )
            C_Coefficients += sum([ Prd_HW[ Relabel-1 ] * e[Node] for Node , Relabel in Relabelling.items() ])
            # 3. Transform the coefficients from basis C (fw's) to B (mixed)
            B_Coefficients = Cartan_Matrix_From_C_To_B * C_Coefficients
            # 4. Collect all coefficients supported over the included nodes (Note: Basis A and B coincide over the included nodes.)
            Prd_Levi_HW = 0*mixed[1]
            Prd_Levi_HW += sum( [  Coefficient*mixed[Node] for Node , Coefficient in enumerate( list(B_Coefficients) , start=1 ) if Node in self.Base_Space().Included_Nodes() ])

            yield self.__class__( Base_Space=self.Base_Space() , Highest_Weight=Prd_Twist_HW+Prd_Levi_HW ) , Prd_Mult


    def __ne__ ( self , other ) :
        """Tests if two objects of the class ``Irreducible_Equivariant_Vector_Bundle_Over_Irreducible_Homogeneous_Variety`` do not coincide."""
        return not self == other


    def __repr__ ( self ) :
        """Returns all attributes which are necessary to initialize the object."""
        return self.Base_Space() , self.Highest_Weight()


    def __str__ ( self ) :
        """Returns a one-line string as short description."""
        return 'VB(' + str( self.Highest_Weight() ) + ')'


    def Base_Space ( self ) :
        """Returns the attribute ``_Base_Space``."""
        return self._Base_Space


    def Coefficients ( self , Basis_Type='B' ) :
        """
        Return the coefficients of the weights with respect to the basis A, B or C.
        (For more details on the different basis types, we refer to the function ``Cartan_Matrix`` of the class ``Parabolic_Subgroup_In_Irreducible_Cartan_Group``.)
        """
        if Basis_Type in [ 'A' , 'alphas' , 'simple roots' ] :
            return self.Base_Space().Cartan_Matrix( From='C' , To='A' ) * self.Coefficients('C')
        elif Basis_Type in [ 'B' , 'mixed' , 'simple roots for included nodes and fundamental weights for excluded nodes' ] :
            return self.Base_Space().Cartan_Matrix( From='C' , To='B' ) * self.Coefficients('C')
        elif Basis_Type in [ 'C' , 'omegas' , 'fundamental weights' ] :
            Vector = vector( QQ , self.Base_Space().Parent_Group().Cartan_Degree() )
            for Node , Coefficient in self.Highest_Weight() : Vector[ Node-1 ] = Coefficient
            return Vector
        else: raise ValueError('The input for the variable ``basis`` is unknown.')


    def Cohomology ( self ) :
        """
        Returns the cohomology.

        INPUT:
        - ``self`` -- VectorBundle_Over_HomogeneousVariety; E over X=G/P.

        OUTPUT:
        - ``Output`` -- Dictionary; Cohomology = { Degree d : H^d( X , E ) }

        ALGORITHM:
        Compute the cohomology by Borel-Weil-Bott, namely Theorem 2.15. in [KP2016].

        Take any λ ∈ P^+_L⊂P_L = P_G (an element of the dominant cone).
        If λ+ρ is G-regular (i.e. does NOT ly on a wall of the Weyl chamber), ...
          ... then there exists a unique w∈WG such that w(λ+ρG) is dominant.
              In this case H^ℓ(w)(X,λ) = WCR( w(λ+ρ)−ρ ) and the other cohomology groups vanish.
        If λ+ρ is G-singular (i.e. lies on a wall of the Weyl chamber), ...
          ... then H^•(X,λ) = 0.

        REFERENCE:
        [KP2016] Kuznetsov, Alexander; Polishchuk, Alexander Exceptional collections on isotropic Grassmannians.
                 J. Eur. Math. Soc. (JEMS) 18 (2016), no. 3, 507–574.
        """
        Rho = sum( self.Base_Space().Parent_Group().Cartan_Type().root_system().weight_space().fundamental_weights() )
        Sum = self.Highest_Weight() + Rho

        if self.Base_Space().Is_Regular( Sum ) :
            [ Shifted_Sum , Direction ] = Sum.to_dominant_chamber(reduced_word=True)
            Degree = len(Direction)
            WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
            CohomologyGroup = WCR( Shifted_Sum - Rho )
            return Degree , CohomologyGroup
        else : return None
    # Synonym for the function ``Cohomology``
    def H ( self ) :
        """Returns the cohomology."""
        return self.Cohomology( )


    def Dual ( self ) :
        """Returns the dual of ``self``."""
        # Dualize the highest weight supported over the twisting part (exluded nodes)
        Dual_Twist_HW = -1 * self.Highest_Weight_Supported_Over( Nodes=self.Base_Space().Parabolic_Subgroup().Excluded_Nodes() ) # Highest Weight supported over the total twisting part

        # Dualize the representation supported over the Levi part (included nodes)
        Dual_Levi_Rep = [ Weyl_Character.dual() for Weyl_Character in self.Representation_Supported_Over_Levi_Part(Decomposition_Style='Fine') ]

        # In the remaining part, we merge the separate dualizations over both the twisting part (excluded nodes) and the Levi part (included nodes)
        # 1. Define subroutine to get coefficients of a given weyl character
        def Get_Coefficients_Of_Weyl_Character ( Weyl_Character ) :
            Simple_Coroots = WeylCharacterRing( Weyl_Character.cartan_type() ).simple_coroots()
            List = []
            for HighestWeight , Multiplicity in Weyl_Character :
                Coroots = [ HighestWeight.inner_product( Element ) for Element in list(Simple_Coroots) ]
                List += Coroots
            return List
        # 2. Write the coefficients of the highest weight of the weyl character ``Dual_Levi_Rep`` in terms of the basis C (fw's)
        n = self.Base_Space().Parent_Group().Cartan_Degree()
        e = { i : vector( QQ , (i-1)*[ 0 ] + [ 1 ] + (n-i)*[ 0 ] ) for i in [ 1 .. n ] }
            # ``e`` is Standard basis to write coefficients of weights in handsome vector-notation and
            # to apply base changes use by multiplication with appropriate Cartan matrix.
        C_Coefficients = vector( QQ , n*[ 0 ] )
        Relabellings = self.Base_Space().Parabolic_Subgroup().Relabelling(Decomposition_Style='Fine')
        for Component_Counter , Weyl_Character in enumerate( Dual_Levi_Rep ) :
            Relabelling = Relabellings[Component_Counter]
            Relabelling_Inverted = { Relabel : Node for Node , Relabel in Relabelling.items() }
            C_Coefficients += sum([ Coefficient * e[ Relabelling_Inverted[Relabel] ] for Relabel , Coefficient in enumerate( Get_Coefficients_Of_Weyl_Character( Weyl_Character ) , start=1 ) ])
        # 3. Transform the coefficients from basis C (fw's) to B (mixed)
        Cartan_Matrix_From_C_To_B = self.Base_Space().Cartan_Matrix( From='C' , To='B' )
        B_Coefficients = Cartan_Matrix_From_C_To_B * C_Coefficients
        # 4. Collect all coefficients supported over the included nodes (Note: Basis A and B coincide over the included nodes.)
        mixed = self.Base_Space().Basis('mixed')
        Dual_Levi_HW = 0*mixed[1]
        Dual_Levi_HW += sum( [  Coefficient*mixed[Node] for Node , Coefficient in enumerate( list(B_Coefficients) , start=1 ) if Node in self.Base_Space().Included_Nodes() ])
        # 5. Finally merging is just adding the weights.
        Dual_HW = Dual_Twist_HW+Dual_Levi_HW

        return self.__class__( Base_Space=self.Base_Space() , Highest_Weight=Dual_HW )


    def Global_Sections ( self ) :
        """Returns the global sections (i.e. the zero-th cohomology)."""
        Cohomology = self.Cohomology()
        if 0 in Cohomology.keys() : return Cohomology[0]
        else : return 0
    # Synonym for the function ``Global_Sections``
    def H0 ( self  ) :
        """Returns the global sections (i.e. the zero-th cohomology)."""
        return self.Global_Sections()

    def Highest_Weight ( self ) :
        """Returns the attribute ``_Highest_Weight``."""
        return self._Highest_Weight


    def Highest_Weight_Supported_Over ( self , Nodes ) :
        """Returns the parts of the highest weights supported over ."""
        All_Nodes = set( self.Base_Space().Parent_Group().Cartan_Type().index_set() )
        assert Nodes.issubset( All_Nodes ) , 'The input for ``Nodes`` must be a subset of all nodes ' + str( All_Nodes ) + '.'
        Coefficients = self.Coefficients('B')
        Basis = self.Base_Space().Basis('mixed')
        return sum( [ Coefficients[Node-1] * Basis[Node] for Node in Nodes ] )


    def Is_SelfDual ( self ) :
        """Tests if `self` is self-dual."""
        return self == self.Dual()


    def Is_Trivial ( self ) :
        """Tests if `self` is trivial."""
        return self.Highest_Weight() == 0


    def Rank ( self ) :
        """Returns the rank of ``self``."""
        return prod([ Representation.degree() for Representation in self.Representation_Supported_Over_Levi_Part( Decomposition_Style='Fine' ) ])


    def Representation_Supported_Over_Levi_Part ( self , Decomposition_Style='Coarse' ) :
        Highest_Weight = self.Highest_Weight_Supported_Over( Nodes=self.Base_Space().Parabolic_Subgroup().Included_Nodes() ) # Highest Weight supported over the total Levi part
        Relabelling = self.Base_Space().Parabolic_Subgroup().Relabelling(Decomposition_Style=Decomposition_Style)

        if Decomposition_Style == 'Fine' :
            # Supported over the irreducible components of the Levi Part
            List = []
            for Component_Counter , Levi_Component in enumerate( self.Base_Space().Parabolic_Subgroup().Initialize_As_Cartan_Group().Components() ) :
                WCR = Levi_Component.Weyl_Character_Ring()
                Basis = Levi_Component.Cartan_Type().root_system().weight_space().fundamental_weights()
                List += [ WCR( sum([ Highest_Weight[Node]*Basis[Relabel] for Node , Relabel in Relabelling[Component_Counter].items() ]) ) ]
            return List

        elif Decomposition_Style == 'Coarse' :
            # Supported over the total Levi Part
            WCR = self.Base_Space().Parabolic_Subgroup().Initialize_As_Cartan_Group().Weyl_Character_Ring()
            Basis = self.Base_Space().Parabolic_Subgroup().Initialize_As_Cartan_Group().Cartan_Type().root_system().weight_space().fundamental_weights()
            return WCR( sum([ Highest_Weight[Node]*Basis[Relabel] for Node , Relabel in Relabelling.items() ]) )

        else : raise ValueError('The input for the decomposition style is unknown.')


    def Support ( self ) :
        """Returns the support of the highest weight."""
        return self.Highest_Weight().support()


    def Weights ( self ) :
        """Returns the weights of ```self``` with are covered by the defining highest weight."""

        # Get all weights cover by the highest weight supported over the twisting part (excluded nodes)
        Weight_Twist = self.Highest_Weight_Supported_Over( Nodes=self.Base_Space().Parabolic_Subgroup().Excluded_Nodes() )

        # Get all weights cover by the highest weight supported over the Levi part (included nodes)
        Rep_Levi = self.Representation_Supported_Over_Levi_Part(Decomposition_Style='Coarse')
        Relabelling = self.Base_Space().Parabolic_Subgroup().Relabelling( Decomposition_Style='Coarse' )
        fw = self.Base_Space().Basis('fundamental weights')
        n = self.Base_Space().Parent_Group().Cartan_Degree()
        e = { i : vector( QQ , (i-1)*[ 0 ] + [ 1 ] + (n-i)*[ 0 ] ) for i in [ 1 .. n ] }
            # ``e`` is Standard basis to write coefficients of weights in handsome vector-notation and
            # to apply base changes use by multiplication with appropriate Cartan matrix.
        Cartan_Matrix_From_C_To_B = self.Base_Space().Cartan_Matrix( From='C' , To='B' )
        mixed = self.Base_Space().Basis('mixed')
        for Weight_Levi_1 , Multiplicity in Rep_Levi.weight_multiplicities().items() :
            Weight_Levi_2 = self.Base_Space().Null_Weight()
            for Node , Relabel in Relabelling.items() :
                Weight_Levi_2 += Weight_Levi_1.to_weight_space().coefficient( Relabel ) * fw[Node]
            # In the remaining part, we merge the separate weights covered by the highest weights of both cases,
            # namely over both the twisting part (excluded nodes) and the Levi part (included nodes)
            # 1. Write the coefficients of the current considered weight in terms of the basis C (fw's)
            C_Coefficients = vector( QQ , n*[0] )
            C_Coefficients += sum([ Coefficient * e[Node] for Node , Coefficient in Weight_Levi_2 ])
            # 2. Transform the coefficients from basis C (fw's) to B (mixed)
            B_Coefficients = Cartan_Matrix_From_C_To_B * C_Coefficients
            # 3. Collect all coefficients supported over the included nodes (Note: Basis A and B coincide over the included nodes.)
            Weight_Levi = 0 * mixed[1]
            Weight_Levi += sum( [  Coefficient*mixed[Node] for Node , Coefficient in enumerate( list(B_Coefficients) , start=1 ) if Node in self.Base_Space().Included_Nodes() ])
            # 4. Form the weight
            Weight = Weight_Levi + Weight_Twist

            yield Weight , Multiplicity


    def Symmetric_Power ( self , p ) :
        """Returns the ``e``-th symmetric power of ``self``."""
        assert p in Integers() , 'The exponent ``e`` must be an integer.'
        if   p < 0 :
            pass
        elif p == 0 :
            fw = self.Base_Space().Basis('fundamental weights')
            yield self.__class__( Base_Space=self.Base_Space() , Highest_Weight=0*fw[1] ) , 1
        else :
            # ... the highest weight supported over the twisting part (excluded nodes)
            Sym_Twist_HW = p * self.Highest_Weight_Supported_Over( Nodes=self.Base_Space().Parabolic_Subgroup().Excluded_Nodes() )
            # ... the highest weight supported over the Levi part (included nodes)
            Sym_Levi_Rep = self.Representation_Supported_Over_Levi_Part(Decomposition_Style='Coarse').symmetric_power(p)

            # In the remaining part, we merge the separate symmetric powers over both the twisting part (excluded nodes) and the Levi part (included nodes)
            # 1. Define subroutine to get coefficients and multiplicities of a given weyl character
            def Get_Coefficients_And_Multiplicity_Of_Weyl_Character ( Weyl_Character ) :
                Simple_Coroots = WeylCharacterRing( Weyl_Character.cartan_type() ).simple_coroots()
                List = []
                for HighestWeight , Multiplicity in Weyl_Character :
                    Coroots = [ HighestWeight.inner_product( Element ) for Element in list(Simple_Coroots) ]
                    List += [ ( Coroots , Multiplicity ) ]
                return List
            # A symmetric power of two highest weights (= irreducible presentations) gives
            # in general a list of highest weights (= reducible presentations)
            n = self.Base_Space().Parent_Group().Cartan_Degree()
            e = { i : vector( QQ , (i-1)*[ 0 ] + [ 1 ] + (n-i)*[ 0 ] ) for i in [ 1 .. n ] }
                # ``e`` is Standard basis to write coefficients of weights in handsome vector-notation and
                # to apply base changes use by multiplication with appropriate Cartan matrix.
            Cartan_Matrix_From_C_To_B = self.Base_Space().Cartan_Matrix( From='C' , To='B' )
            Relabelling = self.Base_Space().Parabolic_Subgroup().Relabelling(Decomposition_Style='Coarse')
            mixed = self.Base_Space().Basis('mixed')
            for Sym_HW , Sym_Mult in Get_Coefficients_And_Multiplicity_Of_Weyl_Character( Sym_Levi_Rep ) :
                # 2. Write the coefficients of the current considered weight in terms of the basis C (fw's)
                C_Coefficients = vector( QQ , n*[ 0 ] )
                C_Coefficients += sum([ Sym_HW[ Relabel-1 ] * e[Node] for Node , Relabel in Relabelling.items() ])
                # 3. Transform the coefficients from basis C (fw's) to B (mixed)
                B_Coefficients = Cartan_Matrix_From_C_To_B * C_Coefficients
                # 4. Collect all coefficients supported over the included nodes (Note: Basis A and B coincide over the included nodes.)
                Sym_Levi_HW = 0*mixed[1]
                Sym_Levi_HW += sum( [  Coefficient*mixed[Node] for Node , Coefficient in enumerate( list(B_Coefficients) , start=1 ) if Node in self.Base_Space().Included_Nodes() ])

                yield self.__class__( Base_Space=self.Base_Space() , Highest_Weight=Sym_Twist_HW+Sym_Levi_HW ) , Sym_Mult


    def Exterior_Power ( self , p ) :
        """Returns the ``e``-th exterior power of ``self``."""
        assert p in Integers() , 'The exponent ``p`` must be an integer.'
        if   p < 0 :
            pass
        elif p == 0 :
            fw = self.Base_Space().Basis('fundamental weights')
            yield self.__class__( Base_Space=self.Base_Space() , Highest_Weight=0*fw[1] ) , 1
        else :
            # ... the highest weight supported over the twisting part (excluded nodes)
            Ext_Twist_HW = p * self.Highest_Weight_Supported_Over( Nodes=self.Base_Space().Parabolic_Subgroup().Excluded_Nodes() )
            # ... the highest weight supported over the Levi part (included nodes)
            Ext_Levi_Rep = self.Representation_Supported_Over_Levi_Part(Decomposition_Style='Coarse').exterior_power(p)

            # In the remaining part, we merge the separate symmetric powers over both the twisting part (excluded nodes) and the Levi part (included nodes)
            # 1. Define subroutine to get coefficients and multiplicities of a given weyl character
            def Get_Coefficients_And_Multiplicity_Of_Weyl_Character ( Weyl_Character ) :
                Simple_Coroots = WeylCharacterRing( Weyl_Character.cartan_type() ).simple_coroots()
                List = []
                for HighestWeight , Multiplicity in Weyl_Character :
                    Coroots = [ HighestWeight.inner_product( Element ) for Element in list(Simple_Coroots) ]
                    List += [ ( Coroots , Multiplicity ) ]
                return List
            # A symmetric power of two highest weights (= irreducible presentations) gives
            # in general a list of highest weights (= reducible presentations)
            n = self.Base_Space().Parent_Group().Cartan_Degree()
            e = { i : vector( QQ , (i-1)*[ 0 ] + [ 1 ] + (n-i)*[ 0 ] ) for i in [ 1 .. n ] }
                # ``e`` is Standard basis to write coefficients of weights in handsome vector-notation and
                # to apply base changes use by multiplication with appropriate Cartan matrix.
            Cartan_Matrix_From_C_To_B = self.Base_Space().Cartan_Matrix( From='C' , To='B' )
            Relabelling = self.Base_Space().Parabolic_Subgroup().Relabelling(Decomposition_Style='Coarse')
            mixed = self.Base_Space().Basis('mixed')
            for Ext_HW , Ext_Mult in Get_Coefficients_And_Multiplicity_Of_Weyl_Character( Ext_Levi_Rep ) :
                # 2. Write the coefficients of the current considered weight in terms of the basis C (fw's)
                C_Coefficients = vector( QQ , n*[ 0 ] )
                C_Coefficients += sum([ Ext_HW[ Relabel-1 ] * e[Node] for Node , Relabel in Relabelling.items() ])
                # 3. Transform the coefficients from basis C (fw's) to B (mixed)
                B_Coefficients = Cartan_Matrix_From_C_To_B * C_Coefficients
                # 4. Collect all coefficients supported over the included nodes (Note: Basis A and B coincide over the included nodes.)
                Ext_Levi_HW = 0*mixed[1]
                Ext_Levi_HW += sum( [  Coefficient*mixed[Node] for Node , Coefficient in enumerate( list(B_Coefficients) , start=1 ) if Node in self.Base_Space().Included_Nodes() ])

                yield self.__class__( Base_Space=self.Base_Space() , Highest_Weight=Ext_Twist_HW+Ext_Levi_HW ) , Ext_Mult